In [123]:
import pandas as pd
import numpy as np

In [124]:
df = pd.read_csv('./green_tripdata_2013-09.csv', skiprows=(1,), index_col=False, parse_dates=[1,2],
                dtype={'VendorID': 'category', 'Store_and_fwd_flag': 'category', 'RateCodeID': 'category', 'Payment_type': 'category'}) \
        .rename(columns={'lpep_pickup_datetime': 'pickup', 'Lpep_dropoff_datetime': 'dropoff'})
df.columns = df.columns.str.strip()

In [125]:
df.head()

,VendorID,pickup,dropoff,Store_and_fwd_flag,RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Passenger_count,Trip_distance,Fare_amount,Extra,MTA_tax,Tip_amount,Tolls_amount,Ehail_fee,Total_amount,Payment_type,Trip_type
0,2,2013-09-01 00:02:00,2013-09-01 00:54:51,N,1,-73.952408,40.810726,-73.983940,40.676285,5,14.35,50.5,0.5,0.5,10.30,0.00,NaN,61.80,1,NaN
1,2,2013-09-01 00:02:34,2013-09-01 00:20:59,N,1,-73.963020,40.711834,-73.966644,40.681690,1,3.24,15.0,0.5,0.5,0.00,0.00,NaN,16.00,2,NaN
2,2,2013-09-01 00:03:06,2013-09-01 00:28:03,N,1,-73.843460,40.755951,-73.989212,40.740528,1,11.27,34.0,0.5,0.5,8.07,5.33,NaN,48.40,1,NaN
3,2,2013-09-01 00:03:30,2013-09-01 00:23:02,N,1,-73.924812,40.754246,-73.978737,40.721504,1,6.63,22.0,0.5,0.5,5.75,0.00,NaN,28.75,1,NaN
4,2,2013-09-01 00:05:12,2013-09-01 00:30:55,N,1,-73.929504,40.756451,-73.856743,40.697037,1,12.84,37.0,0.5,0.5,0.00,0.00,NaN,38.00,1,NaN


In [126]:
df.to_parquet('raw.parquet', engine='pyarrow')

# Derived dataset

- One hot encoding can be done using SQL CASE WHEN which is
long, tedious, repetitive. 

- Same thing apply for int encoding for JFK
- It can be done much faster using pandas.

### below is an example of one hot encoding for pickup hour using sql

```sql
select 
    case when extract(hour from lpep_pickup_datetime)=0 then 1 else 0 end as h_0,
    case when extract(hour from lpep_pickup_datetime)=1 then 1 else 0 end as h_1,
    case when extract(hour from lpep_pickup_datetime)=2 then 1 else 0 end as h_2,
    case when extract(hour from lpep_pickup_datetime)=3 then 1 else 0 end as h_3,
    case when extract(hour from lpep_pickup_datetime)=4 then 1 else 0 end as h_4,
    case when extract(hour from lpep_pickup_datetime)=5 then 1 else 0 end as h_5,
    case when extract(hour from lpep_pickup_datetime)=6 then 1 else 0 end as h_6,
    case when extract(hour from lpep_pickup_datetime)=7 then 1 else 0 end as h_7,
    case when extract(hour from lpep_pickup_datetime)=8 then 1 else 0 end as h_8,
    case when extract(hour from lpep_pickup_datetime)=9 then 1 else 0 end as h_9,
    case when extract(hour from lpep_pickup_datetime)=10 then 1 else 0 end as h_10,
    case when extract(hour from lpep_pickup_datetime)=11 then 1 else 0 end as h_11,
    case when extract(hour from lpep_pickup_datetime)=12 then 1 else 0 end as h_12,
    case when extract(hour from lpep_pickup_datetime)=13 then 1 else 0 end as h_13,
    case when extract(hour from lpep_pickup_datetime)=14 then 1 else 0 end as h_14,
    case when extract(hour from lpep_pickup_datetime)=15 then 1 else 0 end as h_15,
    case when extract(hour from lpep_pickup_datetime)=16 then 1 else 0 end as h_16,
    case when extract(hour from lpep_pickup_datetime)=17 then 1 else 0 end as h_17,
    case when extract(hour from lpep_pickup_datetime)=18 then 1 else 0 end as h_18,
    case when extract(hour from lpep_pickup_datetime)=19 then 1 else 0 end as h_19,
    case when extract(hour from lpep_pickup_datetime)=20 then 1 else 0 end as h_20,
    case when extract(hour from lpep_pickup_datetime)=21 then 1 else 0 end as h_21,
    case when extract(hour from lpep_pickup_datetime)=22 then 1 else 0 end as h_22,
    case when extract(hour from lpep_pickup_datetime)=23 then 1 else 0 end as h_23,
    *
from table;

```

## one-hot encoding using pandas

In [127]:
hour = pd.get_dummies(df.pickup.dt.hour, prefix='hour')

In [128]:
# 0 (Monday) to 6 (Sunday)
weekday = pd.get_dummies(df.pickup.dt.dayofweek, prefix='weekday')

In [129]:
df = pd.concat([df, hour, weekday], axis=1)

## duration of trips in secs

In [130]:
df['duration_seconds'] = (df['dropoff'] - df['pickup']).dt.seconds

## pickup or dropoff at JFK airport

Approach

- use google maps to get approximate coordinate bounding box for JFK
- check whether the pickup or dropoff coordinates lie between the bounding box above by comparing long and lat
- encoding: 0 not related to JFK, 1 pickup at JFK, 2 dropoff at JFK

In [131]:
max_long = -73.773933
min_long = -73.796335
max_lat = 40.652122
min_lat = 40.639553

# 1 => pickup in JFK, 2 => dropoff in JFK
df['from_or_to_jfk'] = np.select(
    [
        (
              (df['Pickup_longitude'] >= min_long) 
            & (df['Pickup_longitude'] <= max_long) 
            & (df['Pickup_latitude'] >= min_lat) 
            & (df['Pickup_latitude'] <= max_lat) 
        ),
        (
              (df['Dropoff_longitude'] >= min_long) 
            & (df['Dropoff_longitude'] <= max_long) 
            & (df['Dropoff_latitude'] >= min_lat) 
            & (df['Dropoff_latitude'] <= max_lat) 
        )
    ],
    [1, 2],
default=0)

In [132]:
df['from_or_to_jfk'].value_counts()

0    49150
2      492
1        5
Name: from_or_to_jfk, dtype: int64

In [133]:
df.head()

,VendorID,pickup,dropoff,Store_and_fwd_flag,RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Passenger_count,...,hour_23,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,duration_seconds,from_or_to_jfk
0,2,2013-09-01 00:02:00,2013-09-01 00:54:51,N,1,-73.952408,40.810726,-73.983940,40.676285,5,...,0,0,0,0,0,0,0,1,3171,0
1,2,2013-09-01 00:02:34,2013-09-01 00:20:59,N,1,-73.963020,40.711834,-73.966644,40.681690,1,...,0,0,0,0,0,0,0,1,1105,0
2,2,2013-09-01 00:03:06,2013-09-01 00:28:03,N,1,-73.843460,40.755951,-73.989212,40.740528,1,...,0,0,0,0,0,0,0,1,1497,0
3,2,2013-09-01 00:03:30,2013-09-01 00:23:02,N,1,-73.924812,40.754246,-73.978737,40.721504,1,...,0,0,0,0,0,0,0,1,1172,0
4,2,2013-09-01 00:05:12,2013-09-01 00:30:55,N,1,-73.929504,40.756451,-73.856743,40.697037,1,...,0,0,0,0,0,0,0,1,1543,0


In [134]:
df.to_parquet('derived.parquet', engine='pyarrow')